In [1]:
import sys
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
from scipy import sparse
import torch
import torch.nn.functional as F
from datetime import date, timedelta
import torch.optim as optim
import torch.nn as nn
from torch_geometric.nn import Sequential, GCNConv, Linear
from torch_geometric import utils, data
from torch_geometric.loader import DataLoader
from sklearn.metrics import r2_score, classification_report
from torch_scatter import scatter
import subprocess
import time
t = time.time()
pd.set_option('mode.chained_assignment',None)
device = torch.device('cuda:1') if torch.cuda.is_available() else torch.device('cpu')
no_days = 2
print(device)

class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=10, verbose=False, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 10
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score:
            self.counter += 1
            if self.counter % 5 == 0:
                self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss


def r2_loss(output, target):
    target_mean = torch.mean(target)
    ss_tot = torch.sum((target - target_mean) ** 2)
    ss_res = torch.sum((target - output) ** 2)
    r2 = 1 - ss_res / ss_tot
    return -r2


batch_size = 512

# Load slicing
with open("data/processed/Sample_NC", "rb") as fp: 
    nc = pickle.load(fp)

with open("data/processed/Sample_CC", "rb") as fp: 
    cc = pickle.load(fp)

# For classification
df_full = pd.read_csv('data/processed/SimpleNNData.csv', index_col=0, parse_dates = [1]).sort_values(by = 'time')
Clas_Coef = dict(pd.concat([df_full.time.dt.hour.iloc[np.concatenate(cc[:2])],df_full.time_to_reservation.iloc[np.concatenate(cc[:2])]], axis = 1).groupby('time')['time_to_reservation'].mean()*2)
df_clas = pd.concat([df_full.time.dt.hour.iloc[cc[2]],df_full.time_to_reservation.iloc[cc[2]]], axis = 1)
df_clas['Cut'] = df_clas.time.map(dict(Clas_Coef))
df_clas = df_clas.iloc[:sum([len(x[2]) for x in nc[:(no_days+1)]])]
zones = [int(z[3:]) for z in df_full.filter(regex = 'lz').columns]
del df_full, cc

# Load weather
Weather_Scale = pd.read_csv('data/processed/MinMaxWeather.csv', index_col=0)


def make_PTG(graph, zones, Weather_Scale):
    attr, adj = graph

    # Filter out 
    if (attr.time_to_reservation.values[-1] >= 48) or ~attr.next_customer[-1]:
        return None
    
    if attr.leave_zone[-1] not in zones:
        return None

    # Slice
    _, labels = sparse.csgraph.connected_components(csgraph=adj, directed=False, return_labels=True)
    newl = labels[-1]
    indices = labels == newl   

    attr = attr[indices]
    adj = adj[indices,:].tocsc()[:,indices].tocsr()

    # drop
    attr.drop(columns=['park_location_lat', 'park_location_long', 'leave_location_lat', 'leave_location_long', 'park_fuel', 'park_zone', 'moved', 'movedTF', 'prev_customer', 'next_customer', 'action'], inplace = True)
    attr.drop(columns = ['leave_zone'], inplace = True)
    # One hot encoding
    attr['engine']= pd.Categorical(attr['engine'], categories=['118I', 'I3', 'COOPER', 'X1'])
    attr = pd.get_dummies(attr, columns = ['engine'], prefix='eng')

    # Add degree
    attr['degree'] = np.squeeze(np.asarray(adj.sum(axis=1)))/50

    # Normalize fuel, weahter and dist 
    attr['leave_fuel'] = attr['leave_fuel']/100
    attr['dist_to_station'] = attr['dist_to_station']/5000
    attr[Weather_Scale.index] = (attr[Weather_Scale.index] - Weather_Scale['Min'])/Weather_Scale['diff']

    if attr.isnull().sum().sum() > 0:
        return attr

    # Get edges
    edge_index, edge_weight = utils.convert.from_scipy_sparse_matrix(adj)

    # Make pytorch data type
    d = data.Data(x = torch.tensor(attr.drop(columns = ['time_to_reservation']).to_numpy(dtype = 'float')).float(), edge_index=edge_index, edge_attr=edge_weight.float(), y = torch.tensor(attr.time_to_reservation.values).float())

    return d

cpu


In [2]:
# Load files
sdate = date(2019, 9, 1) # start date
delta = timedelta(days=2)
files = ['data/processed/Graphs/'+(sdate + timedelta(days=i)).strftime("%Y%m%d")+'.pickle' for i in range(delta.days + 1)]

dataset = []

with open(files[0], 'rb') as f:
    graph_collection = pickle.load(f)

for g in graph_collection.values():
    res = make_PTG(g,zones, Weather_Scale)
    dataset.append(res)
    if type(res) == type(df_clas):
        print('hi')
        break

#train_data = [dataset[i] for i in nc[0][0]]
#val_data = [dataset[i] for i in nc[0][1]]
#test_data = [dataset[i] for i in nc[0][2]]

In [3]:
for file, slicer in tqdm(zip(files[1:], nc[1:len(files)]), total = len(files)-1):
    dataset = []
    with open(file, 'rb') as f:
        graph_collection = pickle.load(f)

    for g in graph_collection.values():
        res = make_PTG(g,zones, Weather_Scale)
        if res:
            dataset.append(res)

    train_data = torch.utils.data.ConcatDataset([train_data,[dataset[i] for i in slicer[0]]])
    val_data = torch.utils.data.ConcatDataset([val_data,[dataset[i] for i in slicer[1]]])
    test_data = torch.utils.data.ConcatDataset([test_data,[dataset[i] for i in slicer[2]]])


train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True, num_workers = 4)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True, drop_last=True, num_workers = 4)
test_loader = DataLoader(test_data, batch_size=1, shuffle=False, drop_last=False, num_workers = 4)

100%|██████████| 2/2 [00:48<00:00, 24.39s/it]


In [26]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.convM = Sequential('x, edge_index, edge_weight', [
        (GCNConv(17,12, aggr = 'max'),'x, edge_index, edge_weight -> x'),
        nn.ReLU(inplace = True),
        (nn.Dropout(0.25), 'x -> x')
        ])

        self.convA = Sequential('x, edge_index, edge_weight', [
        (GCNConv(17,12, aggr = 'add'),'x, edge_index, edge_weight -> x'),
        nn.ReLU(inplace = True),
        (nn.Dropout(0.2), 'x -> x')
        ])

        self.linS = Sequential('x', [
        (Linear(17,12),'x -> x'),
        nn.ReLU(inplace = True),
        (nn.Dropout(0.2), 'x -> x')
        ])

        self.seq = Sequential('x', [
            (Linear(36,16),'x -> x'),
            nn.ReLU(inplace = True),
            (nn.Dropout(0.2), 'x -> x'),
            (Linear(16,1),'x -> x')
        ])


    def forward(self, data):
        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_attr

        xConvM = self.convM(x, edge_index, edge_weight)
        xConvA = self.convA(x, edge_index, edge_weight)
        xLin = self.linS(x)

        x = torch.cat([xConvM,xConvA,xLin], axis = 1)

        x = self.seq(x)

        return x.squeeze()

In [27]:
GNN = GCN().to(device)
print(GNN, sum(p.numel() for p in GNN.parameters()))
print('Start learning')

optimizer = optim.Adam(GNN.parameters(), lr=0.001, weight_decay = 0.00001) #Chaged to Adam and learning + regulariztion rate set

GCN(
  (convM): Sequential(
    (0): GCNConv(17, 12)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.25, inplace=False)
  )
  (convA): Sequential(
    (0): GCNConv(17, 12)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.2, inplace=False)
  )
  (linS): Sequential(
    (0): Linear(17, 12, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.2, inplace=False)
  )
  (seq): Sequential(
    (0): Linear(36, 16, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(16, 1, bias=True)
  )
) 1257
Start learning


In [28]:
# Set number of epochs
num_epochs = 2

# Set up lists for loss/R2
train_r2, train_loss = [], []
valid_r2, valid_loss = [], []
cur_loss = 0
train_losses = []
val_losses = []

no_train = len(train_loader)
no_val = len(val_loader)

for epoch in tqdm(range(num_epochs)):
    ### Train
    cur_loss_train = 0
    GNN.train()
    for batch in train_loader:
        batch.to(device)
        optimizer.zero_grad()
        out = GNN(batch)
        batch_loss = r2_loss(out[batch.ptr[1:]-1],batch.y[batch.ptr[1:]-1])
        batch_loss.backward()
        optimizer.step()

        cur_loss_train += batch_loss.item()
    
    train_losses.append(cur_loss_train/no_train)

    ### Evaluate training
    with torch.no_grad():
        GNN.eval()
        train_preds, train_targs = [], []
        for batch in train_loader:
            target_mask = batch.ptr[1:]-1
            batch.to(device)
            preds = GNN(batch)
            train_targs += list(batch.y.cpu().numpy()[target_mask])
            train_preds += list(preds.cpu().detach().numpy()[target_mask])


    ### Evaluate validation
        val_preds, val_targs = [], []
        cur_loss_val = 0
        for batch in val_loader:
            batch.to(device)
            preds = GNN(batch)[batch.ptr[1:]-1]
            y_val = batch.y[batch.ptr[1:]-1]
            val_targs += list(y_val.cpu().numpy())
            val_preds += list(preds.cpu().detach().numpy())
            cur_loss_val += r2_loss(preds, y_val)

        val_losses.append(cur_loss_val/no_val)


    train_r2_cur = r2_score(train_targs, train_preds)
    valid_r2_cur = r2_score(val_targs, val_preds)
    
    train_r2.append(train_r2_cur)
    valid_r2.append(valid_r2_cur)

    print("Epoch %2i: Train Loss %f, Valid Loss %f, Train R2 %f, Valid R2 %f" % (
                epoch+1, train_losses[-1], val_losses[-1],train_r2_cur, valid_r2_cur))



  0%|          | 0/2 [00:52<?, ?it/s]


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [69]:
batch.x[:7,-7]

tensor([0.0200, 0.0000, 0.0000, 0.0600, 0.0000,    nan, 0.0200])

In [70]:
torch.isnan(batch.x[:,-7]).sum()

tensor(121)

In [35]:
torch.isnan(GNN(batch)[batch.ptr[1:]-1])

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, Tr

In [15]:
train_loader.dataset[1].x

tensor([[ 0.8400,  1.0000, -0.3006,  ...,  0.0000,  0.0000,  0.0901],
        [ 0.1500,  1.0000,  0.6988,  ...,  0.0000,  0.0000,  0.1655],
        [ 0.7300,  1.0000,  0.2284,  ...,  0.0000,  0.0000,  0.3286],
        ...,
        [ 0.6500,  1.0000,  0.8953,  ...,  0.0000,  0.0000,  0.2812],
        [ 0.5600,  1.0000,  0.8922,  ...,  0.0000,  0.0000,  0.3438],
        [ 0.9700,  1.0000,  0.8898,  ...,  0.0000,  0.0000,  0.0410]])

In [80]:
# Load data
import pandas as pd
import networkx as nx
import numpy as np
from tqdm import tqdm
import pickle
from scipy import sparse
import matplotlib.pyplot as plt
import torch
#from torch_geometric import utils, data
 
def haversine_start(df, car1, car2, max_dist = 1500):
    def _edge_weight(x, max_dist):
        return max((max_dist-x)/max_dist,0)
    dfc1 = df[df.car == car1]
    dfc2 = df[df.car == car2]

    point1 = dfc1[['leave_location_lat','leave_location_long']].values[0]
    point2 = dfc2[['leave_location_lat','leave_location_long']].values[0]
    #return point1

    # convert decimal degrees to radians
    lat1, lon1 = map(np.radians, point1)
    lat2, lon2 = map(np.radians, point2)

    # Deltas
    delta_lon = lon2 - lon1 
    delta_lat = lat2 - lat1 
    
    # haversine formula 
    a = np.sin(delta_lat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(delta_lon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    r = 6371000 # Radius of earth in m
    return _edge_weight(c * r, max_dist)

def haversine_add(current_locs, to_add, max_dist = 1500):
    def _edge_weight(x, max_dist):
        return max((max_dist-x)/max_dist,0)
    def _haversine(point1, lat2, lon2):
        
         # convert decimal degrees to radians
        lat1, lon1 = map(np.radians, point1)

        # Deltas
        delta_lon = lon2 - lon1 
        delta_lat = lat2 - lat1 
        
        # haversine formula 
        a = np.sin(delta_lat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(delta_lon/2)**2
        c = 2 * np.arcsin(np.sqrt(a)) 
        r = 6371000 # Radius of earth in m
        return c * r

    leave_lat_add, leave_long_add = to_add.leave_location_lat, to_add.leave_location_long

    lat2, lon2 = map(np.radians, [leave_lat_add,leave_long_add])

    new_weights = [_edge_weight(_haversine(loc, lat2, lon2), max_dist) for loc in current_locs]

    return new_weights

def delete_rc(mat, i):
    # row
    n = mat.indptr[i+1] - mat.indptr[i]
    if n > 0:
        mat.data[mat.indptr[i]:-n] = mat.data[mat.indptr[i+1]:]
        mat.data = mat.data[:-n]
        mat.indices[mat.indptr[i]:-n] = mat.indices[mat.indptr[i+1]:]
        mat.indices = mat.indices[:-n]
    mat.indptr[i:-1] = mat.indptr[i+1:]
    mat.indptr[i:] -= n
    mat.indptr = mat.indptr[:-1]
    mat._shape = (mat._shape[0]-1, mat._shape[1])

    # col
    mat = mat.tocsc()
    n = mat.indptr[i+1] - mat.indptr[i]
    if n > 0:
        mat.data[mat.indptr[i]:-n] = mat.data[mat.indptr[i+1]:]
        mat.data = mat.data[:-n]
        mat.indices[mat.indptr[i]:-n] = mat.indices[mat.indptr[i+1]:]
        mat.indices = mat.indices[:-n]
    mat.indptr[i:-1] = mat.indptr[i+1:]
    mat.indptr[i:] -= n
    mat.indptr = mat.indptr[:-1]
    mat._shape = (mat._shape[0], mat._shape[1]-1)

    return mat.tocsr()

# Haversine function for stations
def haversine_station(point1, point2):
    # convert decimal degrees to radians
    lat1, lon1 = map(np.radians, point1)
    lat2, lon2 = map(np.radians, point2)

    # Deltas
    delta_lon = lon2 - lon1 
    delta_lat = lat2 - lat1 

    # haversine formula 
    a = np.sin(delta_lat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(delta_lon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    r = 6371000 # Radius of earth in m
    return c * r

# Load data
df = pd.read_csv('data/processed/VacancySplit.csv', index_col=0, parse_dates = [2]).astype({'time_to_reservation': 'float32', 'park_location_lat': 'float32', 'park_location_long': 'float32', 'leave_location_lat': 'float32', 'leave_location_long': 'float32', 'park_zone': 'int32', 'leave_zone': 'int32', 'park_fuel': 'int8', 'leave_fuel': 'int8', 'moved': 'float32', 'movedTF': 'bool'})

# Time variables
df['weekend'] = df.time.dt.weekday//5

# Time encoding
def circle_transform(col, max_val=86400):
    tot_sec = ((col - col.dt.normalize()) / pd.Timedelta('1 second')).astype(int)
    cos_val = np.cos(2*np.pi*tot_sec/max_val)
    sin_val = np.sin(2*np.pi*tot_sec/max_val)
    return cos_val, sin_val

df['Time_Cos'], df['Time_Sin'] = [x.values for x in circle_transform(df.time)]

# Join weather
df_weather = pd.read_csv('data/processed/weather.csv', index_col=0, parse_dates=[0])
df['timeH'] = df.time.round('H')
df = df.set_index('timeH').join(df_weather).reset_index(drop=True)

# Create init graph
Start_Time = pd.Timestamp('2019-08-31 21:00:00')
End_Time = pd.Timestamp('2019-11-01 12:00:00')
start_df = df[df.time <= Start_Time]
propegate_df = df[(df.time > Start_Time) & (df.time <= End_Time)]

In [85]:
# Start
CarID_dict_start = dict(iter(start_df.groupby('car')))
Start_Garph_data = []

for sub_df in CarID_dict_start.values():
    last_obs = sub_df.iloc[-1]
    if last_obs.action: # True is park
        Start_Garph_data.append(last_obs)

start_df_graph = pd.DataFrame(Start_Garph_data)